In [2]:
#Importing libraries for BERT(Bidirectional Encoder Representations from Transformers)
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [4]:
# Importing library for data manipulation
import pandas as pd

In [5]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.Category.value_counts()

,count
Category,
ham,4825
spam,747


We can see that there is a category bias, we have to work on it.

This type for ease we will use downscaling and next time we will use SMOTE and others as well.

In [7]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [8]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [9]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [11]:
# Now lets integrate both of them
df_balanced = pd.concat([df_spam, df_ham_downsampled])
df_balanced.shape

(1494, 2)

In [12]:
df_balanced.Category.value_counts()

,count
Category,
spam,747
ham,747


In [13]:
# Nice we got this thing sorted.

In [14]:
df_balanced.head()

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [15]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.head()

,Category,Message,spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",1


In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'], stratify=df_balanced['spam'])

In [17]:
x_train.shape

(1120,)

In [18]:
x_test.shape

(374,)

In [20]:
374/(374+1120)

0.250334672021419

In [21]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [22]:
def get_sentence_embeding(test_text):
  bert_preprocess_model = hub.KerasLayer(preprocess_url)
  bert_model = hub.KerasLayer(encoder_url)

  text_preprocessed = bert_preprocess_model(test_text)
  return bert_model(text_preprocessed)['pooled_output']

get_sentence_embeding(['500$ discount. hurry up','Bhavin, are you up for a volleybal game tomorrow?'])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435168 , -0.5132724 , -0.8884571 , ..., -0.7474884 ,
        -0.7531473 ,  0.91964483],
       [-0.87208354, -0.50543964, -0.94446665, ..., -0.85847497,
        -0.7174535 ,  0.88082975]], dtype=float32)>

In [23]:
e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [24]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[-0.760692  , -0.14219385,  0.49604586, ...,  0.42165312,
        -0.532214  ,  0.8031219 ],
       [-0.86023223, -0.2124296 ,  0.49156848, ...,  0.39797997,
        -0.60506296,  0.8447167 ],
       [-0.7128862 , -0.15463911,  0.384017  , ...,  0.3527875 ,
        -0.5099134 ,  0.734741  ],
       [-0.82533467, -0.35550597, -0.5906975 , ..., -0.01613778,
        -0.61417586,  0.8723029 ],
       [-0.75041336, -0.2681262 , -0.26689708, ...,  0.02839353,
        -0.5938098 ,  0.7974988 ],
       [-0.78544426, -0.2994976 ,  0.41027158, ...,  0.52225286,
        -0.49573594,  0.8150757 ]], dtype=float32)>

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [27]:
cosine_similarity([e[0]],[e[2]])

array([[0.9954098]], dtype=float32)

In [28]:
cosine_similarity([e[1]],[e[2]])

array([[0.9850745]], dtype=float32)

In [29]:
cosine_similarity([e[3]],[e[4]])

array([[0.9872036]], dtype=float32)

In [30]:
cosine_similarity([e[3]],[e[5]])

array([[0.8759415]], dtype=float32)

In [31]:
cosine_similarity([e[5]],[e[4]])

array([[0.9164165]], dtype=float32)

In [33]:
cosine_similarity([e[0]],[e[5]])

array([[0.9571837]], dtype=float32)

In [34]:
cosine_similarity([e[0]],[e[3]])

array([[0.84703827]], dtype=float32)

In [35]:
# Importing libraries for neural networks
import tensorflow as tf

              ENCODER         LAYERS      ACTIVATON
1. BASE       12                768           12
2. LARGE      24                1024          16

In [42]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

In [45]:
# We are suing Functional model this time to practise over Sequential model
"""text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)"""

import keras_nlp

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name ='text')
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased",trainable=True)
encoder_inputs = preprocessor(text_input)
encoder = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased")
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
f = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# Construct a final model
model = tf.keras.Model(inputs=[text_input], outputs=[f])

100%|██████████| 510/510 [00:00<00:00, 634kB/s]


100%|██████████| 547/547 [00:00<00:00, 673kB/s]


100%|██████████| 226k/226k [00:00<00:00, 1.01MB/s]


100%|██████████| 418M/418M [00:09<00:00, 47.9MB/s]


In [46]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text (InputLayer)         │ (None)                 │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_text_classifier_pre… │ [(None, 512), (None,   │              0 │ text[0][0]             │
│ (BertTextClassifierPrepr… │ 512), (None, 512)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 768), (None,   │    109,482,240 │ bert_text_classifier_… │
│ (BertBackbone)            │ 512, 768)]             │                │ bert_text_classifier_… │
│                           │                        │                │ bert_text_classifier_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 768)            │              0 │ bert_backbone[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │            769 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 109,483,009 (417.64 MB)

 Trainable params: 109,483,009 (417.64 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

In [50]:
model.fit(x_train, y_train, epochs=10)

ValueError: Invalid dtype: object

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)